In [1]:
'''with summarize p4 dataset'''

'with summarize p4 dataset'

In [2]:
!pip install sentence_transformers
import numpy as np 
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import os
import re

import nltk
from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models import HdpModel
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e52d5475d8840dbcc5f195cdaa8338f09549fc0d8191eda712b8c798b5549b08
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence_transformers


In [3]:

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
model = SentenceTransformer('all-MiniLM-L6-v2')


In [4]:
expected_answer="""Dear @ORGANIZATION1, The computer blinked to life and an image of a blonde haired girl filled the screen. It was easy to find out how life was in @LOCATION2, thanks to the actual @CAPS1 girl explaining it. Going to the library wouldn't have filled one with this priceless information and human interection. Computers are a nessessity of life if soceity wishes to grow and expand. They should be supported because they teach hand eye coordination, give people the ability to learn about faraway places, and allow people to talk to others online. Firstly, computers help teach hand eye coordination. Hand-eye coordination is a useful ability that is usod to excel in sports. In a recent survey, @PERCENT1 of kids felt their hand eye coordination improves after computer use. Even a simple thing like tying can build up this skill. Famous neurologist @CAPS2 @PERSON1 stated in an article last week that, "@CAPS3 and computer strength the @CAPS2. When on the computer, you automatically process what the eyes see into a command for your hands." @CAPS4 hand eye coordination can improve people in sports such as baseball and basketball. If someone wan't to become better in these sports, all they'd need to do was turn on the computer. Once people become better at sports, they're more likely to play them and become more healthy. In reality, computers can help with exercising instead of decreasing it. Additionaly, computers allow people to access information about faraway places and people. If someone wanted to reasearch @LOCATION1, all they'd need to do was type in a search would be presented to them in it would link forever to search through countless things. Also, having the ability to learn about cultures can make peole peole and their cultures, they understand others something. Increase tolerance people are. Computers are a resourceful tool that they can help people in every different aspect of life. Lastly, computer and in technology can allow people to chat. Computer chat and video chat can help the all different nations. Bring on good terms places other than can help us understand story comes out about something that happend in @LOCATION3, people can just go on their computer and ask an actual @LOCATION3 citizen their take on the matter. Also, video chat and online conversation can cut down on expensive phone bills. No one wants to pay more than they have to in this economy. Another good point is that you can acess family members you scaresly visit. It can help you connect within your own family more. Oviously, computers are a useful aid in todays era. their advancements push the world foreward to a better place. Computers can help people because they help teach handeye coordination, give people the bility to learn about faraway places and people, and allow people to talk online with others. Think of a world with no computers or technologicall advancements. The world would be sectored and unified, contact between people scare, and information even. The internet is like thousands or librarys put together. Nobody would know much about other nations and news would travel slower. Is that the kind of palce you want people to live in?"""
answer="""I think computers are good because you can talk to your friends and family on the computers. People needs computers to look for a job. Some people spend to much time on the computers then on homework people need to stop."""

In [5]:
def preprocess_text(x:str,remove_stopwords=False)->str:
    x = x.lower()
    x = re.sub("[^a-z\s+]","",x)
    if remove_stopwords:
        x = " ".join([word for word in x.split() if word not in  stopwords.words('english')])
    return x
preprocess_text(expected_answer,remove_stopwords=True)


'dear organization computer blinked life image blonde haired girl filled screen easy find life location thanks actual caps girl explaining going library wouldnt filled one priceless information human interection computers nessessity life soceity wishes grow expand supported teach hand eye coordination give people ability learn faraway places allow people talk others online firstly computers help teach hand eye coordination handeye coordination useful ability usod excel sports recent survey percent kids felt hand eye coordination improves computer use even simple thing like tying build skill famous neurologist caps person stated article last week caps computer strength caps computer automatically process eyes see command hands caps hand eye coordination improve people sports baseball basketball someone want become better sports theyd need turn computer people become better sports theyre likely play become healthy reality computers help exercising instead decreasing additionaly computers

In [6]:
def get_similarity_using_HDP(answer,expected_answer):
   
    data = [preprocess_text(temp).split(" ") for temp in [answer,expected_answer]]
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    print(dictionary)
    hdp = HdpModel(corpus, dictionary)
    doc1_topics = hdp[corpus[0]]
    doc2_topics = hdp[corpus[1]]
    similarity = cosine_similarity(doc1_topics, doc2_topics).mean()
   
    return (similarity)
similarity = get_similarity_using_HDP(answer,expected_answer)
similarity

Dictionary(253 unique tokens: ['a', 'and', 'are', 'because', 'can']...)


0.8498256193401008

In [7]:
def get_similarity_using_bert(answer,expected_answer):
    
    data = [preprocess_text(temp) for temp in [answer,expected_answer]]
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    embeddings = model.encode(data)
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]]).mean()
    return (similarity)
similarity = get_similarity_using_bert(answer,expected_answer)
similarity


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.74337614

In [8]:
from gensim.models import LdaModel
def get_similarity_using_lda(answer,expected_answer):
    temp = [preprocess_text(x,remove_stopwords=True).split(" ") for x in [answer,expected_answer]]
    dictionary = Dictionary(temp)
    matrix = [dictionary.doc2bow(doc) for doc in temp]
    # training the LDA model
    num_topics = 20
    ldamodel = LdaModel(matrix, num_topics = num_topics, id2word = dictionary, passes=50)
    sentence1_topic_distribution = ldamodel[dictionary.doc2bow(temp[0])]
    sentence2_topic_distribution = ldamodel[dictionary.doc2bow(temp[1])]
    similarity = cosine_similarity(sentence1_topic_distribution,sentence2_topic_distribution).mean()
    return (similarity)
get_similarity_using_lda(answer,expected_answer)

0.997319987144975

In [11]:
def get_similarity_using_lsa(answer,expected_answer):
    
    documents = [preprocess_text(x,remove_stopwords=True) for x in [answer, expected_answer]]

    # Create the TF-IDF matrix
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Perform LSA
    lsa = TruncatedSVD(n_components=20)
    doc_topic_matrix = lsa.fit_transform(tfidf_matrix)

    # Measure the similarity between the two documents
    similarity = cosine_similarity(doc_topic_matrix[0].reshape(1,-1), doc_topic_matrix[1].reshape(1,-1))
    return (similarity)
similarity = get_similarity_using_lsa(answer,expected_answer)
similarity

array([[0.32546585]])

In [12]:
df = pd.read_csv('/kaggle/input/answerscript4/p4.csv')


In [13]:
looper = list(df['essay_set'].unique())

df2=df[['essay_set','domain1_score']]
df2['new_score']=None

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
df2['score_LDA']=None
df2['score_LSA']=None
df2['score_HDP']=None
df2['score_bert']=None

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [15]:
def scoring(answer_sheet, students_answers, marks_org):
  c=0
  t1=answer_sheet
  

  data = summarizer(answer_sheet)
  data = data[0]['summary_text']
  t2=data
  embeddings1 = model.encode(data, convert_to_tensor=True)
  t=max(marks_org)
  for enum,i in enumerate(students_answers):
     
     
     embeddings2 = model.encode(i, convert_to_tensor=True)
     cosine_scores = util.cos_sim(embeddings1, embeddings2)
     k=float(t*cosine_scores[0][0])
     df2.at[enum, "new_score"] = k
    # print(i,"-------------------", marks_org[enum], str.format("{0:.2f}", cosine_scores[0][0]*100))
     print(marks_org[enum],k)       
     
    
    
    
    

In [16]:
def scoring5(answer_sheet, students_answers, marks_org):
  c=0
  t1=answer_sheet
  

  data = summarizer(answer_sheet)
  data = data[0]['summary_text']
  t2=data
  
  t=max(marks_org)
  for enum,i in enumerate(students_answers):
     
     k1=get_similarity_using_bert(i,data)
     k=float(t*k1)
     df2.at[enum, "score_bert"] = k
    # print(i,"-------------------", marks_org[enum], str.format("{0:.2f}", cosine_scores[0][0]*100))
    
    
    

In [17]:
def scoring2(answer_sheet, students_answers, marks_org):
  data = summarizer(answer_sheet)
  data = data[0]['summary_text']
  embeddings1 = model.encode(data, convert_to_tensor=True)
  t=max(marks_org)  
  for enum,i in enumerate(students_answers):
   t1=get_similarity_using_lda(i,data)
   k=float(t*t1)
   df2.at[enum, "score_LDA"] = k

In [18]:
def scoring3(answer_sheet, students_answers, marks_org):
  data = summarizer(answer_sheet)
  data = data[0]['summary_text']
  embeddings1 = model.encode(data, convert_to_tensor=True)
  t=max(marks_org)  
  for enum,i in enumerate(students_answers):
   t1=get_similarity_using_lsa(i,data)
   k=float(t*t1)
   df2.at[enum, "score_LSA"] = k


In [19]:
def scoring4(answer_sheet, students_answers, marks_org):
  data = summarizer(answer_sheet)
  data = data[0]['summary_text']
  embeddings1 = model.encode(data, convert_to_tensor=True)
  t=max(marks_org)  
  for enum,i in enumerate(students_answers):
   t1=get_similarity_using_HDP(i,data)
   k=float(t*t1)
   df2.at[enum, "score_HDP"] = k


In [20]:
for i in looper:

     df_ind = df.loc[df['essay_set']==int(i)]

     df_ind = df_ind[['essay','domain1_score']]

     df_ind = df_ind.sort_values(by= ["domain1_score"], ascending=False)

     answer_sheet = df_ind.iloc[0]['essay']

     df_ind = df_ind.drop(0)
    
     students_answers = list(df['essay'].values)
     marks_org = list(df['domain1_score'].values)
     
     scoring(answer_sheet, students_answers, marks_org)
     scoring2(answer_sheet, students_answers, marks_org)
     scoring3(answer_sheet, students_answers, marks_org)
     scoring4(answer_sheet, students_answers, marks_org)
     scoring5(answer_sheet, students_answers, marks_org)   
        
        

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.0 0.8694392442703247


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.0 1.046736717224121


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
import math
MSE = np.square(np.subtract(df2['domain1_score'],df2['score_bert'])).mean()      
rsme = math.sqrt(MSE)  
print("Root Mean Square Error for bert:\n")  
print(rsme)  

Root Mean Square Error for bert:

1.0474511533563793


In [22]:
MSE = np.square(np.subtract(df2['domain1_score'],df2['new_score'])).mean()      
rsme = math.sqrt(MSE)  
print("Root Mean Square Error for bert2:\n")  
print(rsme)  

Root Mean Square Error for bert2:

0.8293908628439393


In [23]:
MSE = np.square(np.subtract(df2['domain1_score'],df2['score_LSA'])).mean()      
rsme = math.sqrt(MSE)  
print("Root Mean Square Error for LSA:\n")  
print(rsme)  

Root Mean Square Error for LSA:

1.3313912888017847


In [24]:
MSE = np.square(np.subtract(df2['domain1_score'],df2['score_LDA'])).mean()      
rsme = math.sqrt(MSE)  
print("Root Mean Square Error for LDA:\n")  
print(rsme)  

Root Mean Square Error for LDA:

1.7177605049196403


In [25]:
MSE = np.square(np.subtract(df2['domain1_score'],df2['score_HDP'])).mean()      
rsme = math.sqrt(MSE)  
print("Root Mean Square Error for HDP:\n")  
print(rsme)  

Root Mean Square Error for HDP:

1.2806473957706814


In [26]:
df2.to_csv('p4_results.csv')

In [27]:
df2.head()

,essay_set,domain1_score,new_score,score_LDA,score_LSA,score_HDP,score_bert
0,4,0.0,0.869439,2.931809,0.199762,2.551784,1.876782
1,4,0.0,1.046737,2.998796,0.0,1.661703,1.463052
2,4,3.0,2.215415,2.999114,0.715585,2.546471,2.301979
3,4,2.0,1.569092,2.998496,0.363573,2.001549,1.956643
4,4,2.0,2.024979,2.999078,0.595685,2.549192,2.173633


In [28]:
'''df["LSA Score"] = df.apply(lambda row: get_similarity_using_lsa(row['essay'],expected_answer,marks_org),axis=1 )
df.to_csv("out.csv",index=False)
df.head()'''

'df["LSA Score"] = df.apply(lambda row: get_similarity_using_lsa(row[\'essay\'],expected_answer,marks_org),axis=1 )\ndf.to_csv("out.csv",index=False)\ndf.head()'

In [29]:
'''df["LDA Score"] = df.apply(lambda row: get_similarity_using_lda(row['essay'],expected_answer,marks_org),axis=1 )
df.to_csv("out.csv",index=False)
df.head()'''

'df["LDA Score"] = df.apply(lambda row: get_similarity_using_lda(row[\'essay\'],expected_answer,marks_org),axis=1 )\ndf.to_csv("out.csv",index=False)\ndf.head()'

In [30]:
'''df["BERT Score"]= df.apply(lambda row: get_similarity_using_bert(row['essay'],expected_answer,marks_org),axis=1)
df.to_csv("out.csv",index=False)
df.head()'''

'df["BERT Score"]= df.apply(lambda row: get_similarity_using_bert(row[\'essay\'],expected_answer,marks_org),axis=1)\ndf.to_csv("out.csv",index=False)\ndf.head()'

In [31]:
'''df["HDP score"]= df.apply(lambda row: get_similarity_using_HDP(row['essay'],expected_answer,marks_org),axis=1)
df.to_csv("out.csv",index=False)
df.head()'''

'df["HDP score"]= df.apply(lambda row: get_similarity_using_HDP(row[\'essay\'],expected_answer,marks_org),axis=1)\ndf.to_csv("out.csv",index=False)\ndf.head()'